In [1]:
library(plyr)
library(sdmpredictors)
library(tidyterra)
library(tidyverse)
library(ggplot2)
library(raster)
library(terra)
library(caret)
library(ade4)

#https://www.ecologi.st/spatial-r/raster-gis-operations-in-r-with-terra.html


Attaching package: 'tidyterra'


The following objects are masked from 'package:plyr':

    arrange, count, mutate, rename, summarise, summarize


The following object is masked from 'package:stats':

    filter


-- Attaching packages --------------------------------------------------------------- tidyverse 1.3.2 --
v ggplot2 3.4.0      v purrr   0.3.5 
v tibble  3.1.8      v dplyr   1.0.10
v tidyr   1.2.1      v stringr 1.5.0 
v readr   2.1.3      v forcats 0.5.2 
-- Conflicts ------------------------------------------------------------------ tidyverse_conflicts() --
x dplyr::arrange()   masks tidyterra::arrange(), plyr::arrange()
x purrr::compact()   masks plyr::compact()
x dplyr::count()     masks tidyterra::count(), plyr::count()
x dplyr::failwith()  masks plyr::failwith()
x dplyr::filter()    masks tidyterra::filter(), stats::filter()
x dplyr::id()        masks plyr::id()
x dplyr::lag()       masks stats::lag()
x dplyr::mutate()    masks tidyterra::mutate(), plyr::mutate()
x dpl

In [2]:
(Bluecarbon_sites <- read.csv("data//BC/BCsampling_sites.tsv", sep ="\t", header=T) %>% 
                     dplyr::mutate(Site = ifelse(Site =="Site1", "Upper", "Lower")) %>%
                     dplyr::rename(estuary = Estuary) %>%
                     dplyr::rename(Biogeographical_region = Biogeographical.region))

estuary,Site,x,y,date,Biogeographical_region
<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
Berg,Upper,18.19648,-32.81314,18 06 2021,Cool Temperate
Berg,Lower,18.14189,-32.77216,18 06 2021,Cool Temperate
Breede,Lower,20.84452,-34.39693,25 06 2021,Warm temperate
Breede,Upper,20.80405,-34.40552,25 06 2021,Warm temperate
Knysna,Upper,23.05105,-34.06119,30 05 2021,Warm temperate
Knysna,Lower,22.99216,-34.03369,30 05 2021,Warm temperate
Olifants,Lower,18.18882,-31.69302,05 07 2021,Cool Temperate
Olifants,Upper,18.19545,-31.66071,05 07 2021,Cool Temperate
Swartkops,Lower,25.59069,-33.83784,25 05 2021,Warm temperate


In [3]:
ZA_admin_st <- geodata::gadm("ZA", path = "data/sdm/")
ZA_admin_sf = sf::st_as_sf(ZA_admin_st)
ZA_extent <- raster::extent(10, 40, -37, -22)

In [4]:
get_raster <- function(path, pattern){

    file_ls <- Sys.glob(file.path(path , pattern))
    raster <- raster::stack(file_ls)
    
    return(raster)
} 

In [5]:
gmed_raster <- get_raster("GMED/*/", "*.asc")
dim(gmed_raster)

[1] 1784 4337   54

In [6]:
gmed_cropped <- raster::crop(gmed_raster, ZA_extent) 
dim(gmed_cropped)

[1] 181 362  54

In [7]:
crs(gmed_cropped)

Coordinate Reference System:
Deprecated Proj.4 representation: +proj=longlat +datum=WGS84 +no_defs 
WKT2 2019 representation:
GEOGCRS["unknown",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]],
        ID["EPSG",6326]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433],
        ID["EPSG",8901]],
    CS[ellipsoidal,2],
        AXIS["longitude",east,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433,
                ID["EPSG",9122]]],
        AXIS["latitude",north,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433,
                ID["EPSG",9122]]]] 

In [29]:
NA_cols <- function(data){
            dplyr::select(data ,where(function(x) any(is.na(x))))
}

In [33]:
NA_vars <- as.data.frame(raster::extract(gmed_cropped, Bluecarbon_sites[c("x", "y")])) %>%
           NA_cols()

dim(NA_vars)

[1] 12 54

In [90]:
get_approx_NA <-  function(xy, env_layer, max_dist){
    
    env_value <- raster::extract(env_layer, xy)
    if (is.na(env_value)){
        raster_dist <- replace(distanceFromPoints(env_layer, xy), is.na(env_layer), NA)
        dist_index <- which.min(raster_dist)
        dist_value <- raster_dist@data@values[dist_index]
        approx_value <- if(dist_value > max_dist) NA else  env_layer@data@values[dist_index]
        approx_df <- data.frame(x = xy['x'], y = xy['y'], max_dist, dist_value, approx_value,)
        names(approx_df)[3:4] <- c(paste('dist', names(env_layer), sep = '_'), names(env_layer))
    {
     else{
          
        approx_df <- data.frame(x = xy['x'], y = xy['y'], max_dist, approx_value, dist_value = 0)
        names(approx_df)[3:4] <- c(paste('dist', names(env_layer), sep = '_'), names(env_layer))
      
     }  
    
return(approx_df)  
    
}

ERROR: Error in parse(text = x, srcfile = src): <text>:12:6: unexpected 'else'
11:     {
12:      else
         ^


In [83]:
if is()

 logi NA


In [ ]:
if(dist_value > max_dist) NA else  env_layer@data@values[dist_index]

In [72]:
proxim_var <- function(layer_name, coord_df, raster_stack, max_dist){
    
    env_layer <- raster_stack[[layer_name]]
    proxim_na <- do.call(rbind, apply(X = coord_df, MARGIN = 1, FUN = get_approx_NA, env_layer, max_dist))

return(proxim_na)
}

In [52]:
proxim_df_ls <- lapply(names(gmed_cropped), proxim_var, coord_df = Bluecarbon_sites[c('x','y')], raster_stack = gmed_cropped, max_dist = 100000)
proxim_df <- join_all(proxim_df_ls, by = c('x','y'), type ="full")
NA_vars2 <- proxim_df %>% NA_cols()
dim(NA_vars)
dim(NA_vars  %>% na.omit)

[1] 12 54

[1]  0 54

In [79]:
get_approx_NA(data.frame(x = 18.19648, y = -32.81314), gmed_cropped$bedtemp,  max_dist =  10000)

 num 10000


x,y,dist_bedtemp,bedtemp
<dbl>,<dbl>,<dbl>,<lgl>
18.19648,-32.81314,3800.78,NA


In [63]:
proxim_df

x,y,dist_bedtemp,bedtemp,dist_bnitrate,bnitrate,dist_bo2dissolve,bo2dissolve,dist_bo2utilized,bo2utilized,...,dist_sstrange,sstrange,dist_surcurrent,surcurrent,dist_tideaverage,tideaverage,dist_waveheight,waveheight,dist_windspeed,windspeed
<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,<lgl>,<dbl>,<lgl>,<dbl>,<lgl>,...,<dbl>,<lgl>,<dbl>,<lgl>,<dbl>,<lgl>,<dbl>,<lgl>,<dbl>,<lgl>
18.19648,-32.81314,3800.774,NA,3800.774,NA,3800.774,NA,3800.774,NA,...,3800.774,NA,3800.774,NA,7049.220,NA,3800.774,NA,3800.774,NA
18.14189,-32.77216,3126.634,NA,3126.634,NA,3126.634,NA,3126.634,NA,...,3126.634,NA,3126.634,NA,3126.634,NA,3126.634,NA,3126.634,NA
20.84452,-34.39693,6043.536,NA,6043.536,NA,6043.536,NA,6043.536,NA,...,6043.536,NA,6043.536,NA,6043.536,NA,6043.536,NA,6043.536,NA
20.80405,-34.40552,9496.367,NA,9496.367,NA,9496.367,NA,9496.367,NA,...,9496.367,NA,9496.367,NA,9496.367,NA,9496.367,NA,9496.367,NA
23.05105,-34.06119,2671.226,NA,2671.226,NA,2671.226,NA,2671.226,NA,...,2671.226,NA,2671.226,NA,2671.226,NA,2671.226,NA,2671.226,NA
22.99216,-34.03369,5485.135,NA,5485.135,NA,5485.135,NA,5485.135,NA,...,5485.135,NA,5485.135,NA,5485.135,NA,5485.135,NA,5485.135,NA
18.18882,-31.69302,2793.013,NA,2793.013,NA,2793.013,NA,2793.013,NA,...,2793.013,NA,2793.013,NA,2793.013,NA,2793.013,NA,2793.013,NA
18.19545,-31.66071,3086.375,NA,3086.375,NA,3086.375,NA,3086.375,NA,...,3086.375,NA,3086.375,NA,3086.375,NA,3086.375,NA,3086.375,NA
25.59069,-33.83784,9740.741,NA,9740.741,NA,9740.741,NA,9740.741,NA,...,9740.741,NA,9740.741,NA,9740.741,NA,9740.741,NA,9740.741,NA


In [12]:
fill_envdata <- function(my_layer, env_layers, coord_df, proxim_data){
    
    env_layer <- env_layers[[my_layer]]
    for (i in 1: nrow(Bluecarbon_sites)){
        xy <- coord_df[i,c('x','y')]
        env_value <- raster::extract(env_layer, xy)
        if(is.na(env_value) & !missing(proxim_data)){
            proxim_value <- proxim_df[proxim_df$x == coord_df[i,]$x & proxim_df$y ==  coord_df[i,]$y, my_layer]
            coord_df[i, my_layer] <- proxim_value
        }
        else{
            coord_df[i, my_layer] <- env_value  
        }
    }   
return(coord_df)
}

In [13]:
# #Testing filling of NAs 
# #https://stackoverflow.com/questions/27562076/if-raster-value-na-search-and-extract-the-nearest-non-na-pixel
# set.seed(2)
# # create a 10x10 raster
# r <- raster(ncol=10,nrow=10, xmn=0, xmx=10, ymn=0,ymx=10)
# r[] <- 1:10
# r[sample(1:ncell(r), size = 25)] <- NA
# # plot the raster
# plot(r, axes=F, box=F)
# segments(x0 = 0, y0 = 0:10, x1 = 10, y1 = 0:10, lty=2)
# segments(y0 = 0, x0 = 0:10, y1 = 10, x1 = 0:10, lty=2)
# # create sample points and add them to the plot
# xy = data.frame(x=runif(10,1,10), y=runif(10,1,10))
# points(xy, pch=3)
# text(x = xy$x, y = xy$y, labels = as.character(1:nrow(xy)), pos = 4, cex = 0.7, xpd = NA)

In [14]:
# sampled = apply(X = xy, MARGIN = 1, FUN = function(xy) r@data@values[which.min(replace(distanceFromPoints(r, xy), is.na(r), NA))])
# extracted = extract(x = r, y = xy)
# (sampled_df <- data.frame(xy, extracted, sampled))

In [15]:
#testing the implementation
# test_df <- do.call(rbind, apply(X = xy, MARGIN = 1, FUN = get_approx_NA, env_layer = r, max_dist = 1000000))

In [16]:
# sampled_df$sampled == test_df$layer

In [58]:
filled_env_dfs <- lapply(names(gmed_cropped), fill_envdata, env_layers = gmed_cropped, coord_df = Bluecarbon_sites, proxim_data = proxim_df)

In [55]:
sel_cols <- c("estuary","Site","x","y","date","Biogeographical_region")
env_df <- join_all(filled_env_dfs, by = sel_cols, type ="full")

In [59]:
filled_env_dfs

estuary,Site,x,y,date,Biogeographical_region,bedtemp
<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
Berg,Upper,18.19648,-32.81314,18 06 2021,Cool Temperate,14.97160
Berg,Lower,18.14189,-32.77216,18 06 2021,Cool Temperate,14.97160
Breede,Lower,20.84452,-34.39693,25 06 2021,Warm temperate,NA
Breede,Upper,20.80405,-34.40552,25 06 2021,Warm temperate,NA
Knysna,Upper,23.05105,-34.06119,30 05 2021,Warm temperate,15.56181
Knysna,Lower,22.99216,-34.03369,30 05 2021,Warm temperate,NA
Olifants,Lower,18.18882,-31.69302,05 07 2021,Cool Temperate,11.28881
Olifants,Upper,18.19545,-31.66071,05 07 2021,Cool Temperate,11.28881
Swartkops,Lower,25.59069,-33.83784,25 05 2021,Warm temperate,NA


In [56]:
env_df

estuary,Site,x,y,date,Biogeographical_region,bedtemp,bnitrate,bo2dissolve,bo2utilized,...,sst_mayoct,sst_novapr,sstmax,sstmean,sstmin,sstrange,surcurrent,tideaverage,waveheight,windspeed
<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Berg,Upper,18.19648,-32.81314,18 06 2021,Cool Temperate,14.97160,6.56227,5.401511,0.369455,...,NA,NA,16.98092,15.00000,13.69698,3.000000,0.009436653,NA,4.207474,0.7635055
Berg,Lower,18.14189,-32.77216,18 06 2021,Cool Temperate,14.97160,6.56227,5.401511,0.369455,...,16.04738,NA,16.98092,15.00000,13.69698,3.000000,0.009436653,0.1848701,4.207474,0.7635055
Breede,Lower,20.84452,-34.39693,25 06 2021,Warm temperate,NA,NA,NA,NA,...,18.80169,NA,NA,NA,NA,NA,NA,NA,NA,NA
Breede,Upper,20.80405,-34.40552,25 06 2021,Warm temperate,NA,NA,NA,NA,...,18.80169,NA,NA,NA,NA,NA,NA,NA,NA,NA
Knysna,Upper,23.05105,-34.06119,30 05 2021,Warm temperate,15.56181,15.27581,4.562259,1.181571,...,19.94017,19.9271,21.67366,18.67366,16.00000,4.923164,-0.011619920,0.8444477,7.000000,7.9088192
Knysna,Lower,22.99216,-34.03369,30 05 2021,Warm temperate,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Olifants,Lower,18.18882,-31.69302,05 07 2021,Cool Temperate,11.28881,16.46212,4.113858,2.060184,...,15.49737,NA,17.00000,15.00000,13.00000,3.650872,0.001738676,0.2083865,4.000000,0.7066119
Olifants,Upper,18.19545,-31.66071,05 07 2021,Cool Temperate,11.28881,16.46212,4.113858,2.060184,...,15.49737,NA,17.00000,15.00000,13.00000,3.650872,0.001738676,0.2083865,4.000000,0.7066119
Swartkops,Lower,25.59069,-33.83784,25 05 2021,Warm temperate,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [49]:
Missing_cols <- env_df %>% 
                NA_cols() 
dim(Missing_cols)
dim(Missing_cols %>% na.omit)

[1] 12 54

[1]  0 54

In [50]:
Missing_cols

bedtemp,bnitrate,bo2dissolve,bo2utilized,bphosphate,bsalin,bsalin21k,bsilicate,btemp,btemp21k,...,sst_mayoct,sst_novapr,sstmax,sstmean,sstmin,sstrange,surcurrent,tideaverage,waveheight,windspeed
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
14.97160,6.56227,5.401511,0.369455,0.8635866,NA,NA,7.599373,NA,11.74407,...,NA,NA,16.98092,15.00000,13.69698,3.000000,0.009436653,NA,4.207474,0.7635055
14.97160,6.56227,5.401511,0.369455,0.8635866,NA,34.95545,7.599373,NA,11.74407,...,16.04738,NA,16.98092,15.00000,13.69698,3.000000,0.009436653,0.1848701,4.207474,0.7635055
NA,NA,NA,NA,NA,NA,35.49804,NA,NA,18.02030,...,18.80169,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,35.49804,NA,NA,18.02030,...,18.80169,NA,NA,NA,NA,NA,NA,NA,NA,NA
15.56181,15.27581,4.562259,1.181571,0.8103373,NA,35.31949,10.791760,NA,16.51494,...,19.94017,19.9271,21.67366,18.67366,16.00000,4.923164,-0.011619920,0.8444477,7.000000,7.9088192
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
11.28881,16.46212,4.113858,2.060184,1.4847380,7.473438,34.93443,14.820160,NA,11.56553,...,15.49737,NA,17.00000,15.00000,13.00000,3.650872,0.001738676,0.2083865,4.000000,0.7066119
11.28881,16.46212,4.113858,2.060184,1.4847380,7.473438,34.93443,14.820160,NA,11.56553,...,15.49737,NA,17.00000,15.00000,13.00000,3.650872,0.001738676,0.2083865,4.000000,0.7066119
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [20]:
dplyr::sample_n(env_df, 5)

estuary,Site,x,y,date,Biogeographical_region,bedtemp,bnitrate,bo2dissolve,bo2utilized,...,sst_mayoct,sst_novapr,sstmax,sstmean,sstmin,sstrange,surcurrent,tideaverage,waveheight,windspeed
<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Swartkops,Lower,25.59069,-33.83784,25 05 2021,Warm temperate,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Olifants,Lower,18.18882,-31.69302,05 07 2021,Cool Temperate,11.28881,16.46212,4.113858,2.060184,...,15.49737,NA,17.00000,15.00000,13.00000,3.650872,0.001738676,0.2083865,4.000000,0.7066119
Swartkops,Upper,25.62139,-33.86114,25 05 2021,Warm temperate,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Berg,Upper,18.19648,-32.81314,18 06 2021,Cool Temperate,14.97160,6.56227,5.401511,0.369455,...,NA,NA,16.98092,15.00000,13.69698,3.000000,0.009436653,NA,4.207474,0.7635055
Mngazana,Upper,29.42223,-31.69004,11 10 2021,Subtropical,13.55985,10.43052,4.327785,1.524787,...,NA,NA,24.34826,22.11611,20.10154,4.000000,-0.101078197,0.7961749,4.000000,0.0000000


In [36]:
env_df

estuary,Site,x,y,date,Biogeographical_region,bedtemp,bnitrate,bo2dissolve,bo2utilized,...,sst_mayoct,sst_novapr,sstmax,sstmean,sstmin,sstrange,surcurrent,tideaverage,waveheight,windspeed
<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Berg,Upper,18.19648,-32.81314,18 06 2021,Cool Temperate,14.97160,6.56227,5.401511,0.369455,...,NA,NA,16.98092,15.00000,13.69698,3.000000,0.009436653,NA,4.207474,0.7635055
Berg,Lower,18.14189,-32.77216,18 06 2021,Cool Temperate,14.97160,6.56227,5.401511,0.369455,...,16.04738,NA,16.98092,15.00000,13.69698,3.000000,0.009436653,0.1848701,4.207474,0.7635055
Breede,Lower,20.84452,-34.39693,25 06 2021,Warm temperate,NA,NA,NA,NA,...,18.80169,NA,NA,NA,NA,NA,NA,NA,NA,NA
Breede,Upper,20.80405,-34.40552,25 06 2021,Warm temperate,NA,NA,NA,NA,...,18.80169,NA,NA,NA,NA,NA,NA,NA,NA,NA
Knysna,Upper,23.05105,-34.06119,30 05 2021,Warm temperate,15.56181,15.27581,4.562259,1.181571,...,19.94017,19.9271,21.67366,18.67366,16.00000,4.923164,-0.011619920,0.8444477,7.000000,7.9088192
Knysna,Lower,22.99216,-34.03369,30 05 2021,Warm temperate,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Olifants,Lower,18.18882,-31.69302,05 07 2021,Cool Temperate,11.28881,16.46212,4.113858,2.060184,...,15.49737,NA,17.00000,15.00000,13.00000,3.650872,0.001738676,0.2083865,4.000000,0.7066119
Olifants,Upper,18.19545,-31.66071,05 07 2021,Cool Temperate,11.28881,16.46212,4.113858,2.060184,...,15.49737,NA,17.00000,15.00000,13.00000,3.650872,0.001738676,0.2083865,4.000000,0.7066119
Swartkops,Lower,25.59069,-33.83784,25 05 2021,Warm temperate,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [21]:
gmed_data <- env_df %>% 
             dplyr::select(-c("estuary", "Site", "x", "y", "date", "Biogeographical_region")) %>%
             dplyr::select(-all_of(NA_cols))

In [35]:
cor_matrix <- cor(gmed_data) %>% 
                 data.frame() %>% 
                 dplyr::select(where(function(x) (sum(!is.na(x)) > 1))) %>% 
                 na.omit

In [34]:
cor_matrix

<0 x 0 matrix>

In [24]:
best_gmed_colnames <- caret::findCorrelation(cor_matrix, cutoff = .6, exact = FALSE, names = TRUE)

In [25]:
dim(gmed_data)
gmed_low_cor <- gmed_data %>% 
                dplyr::select(all_of(best_gmed_colnames))
dim(gmed_low_cor)

[1] 12  0

[1] 12  0

In [26]:
fit1 <- lm(MgC_perHa ~ WC_bio2 + WC_bio14 + WC_bio15 + WC_bio18, data = ZAF_bluecarbon_final)
vif_out <-  as.data.frame(car::vif(fit1)) %>% tibble::rownames_to_column(var = "layer_code")
colnames(vif_out)[2] <- "VIF"
(final_layers <- merge(vif_out, best_WC) %>% filter(VIF < 10)) %>% 
                 dplyr::select(layer_code, name, description, description, start_year, end_year, VIF) %>%
                 dplyr::mutate(VIF = round(VIF,2))
print(xtable(final_layers, digits=2, type = "latex"), file = "Supplementary_material/Worldclim_data.tex", include.rownames=FALSE)

ERROR: Error in is.data.frame(data): object 'ZAF_bluecarbon_final' not found


In [ ]:
layers_correlation(final_layers$layer_code)

In [ ]:
ZAF_bluecarbon_final <- data.frame(ZAF_bluecarbon_raw)
for (layer_name in final_layers$layer_code){ 
    
    layer <- env_layers[[layer_name]]
    
    ZAF_bluecarbon_final[layer_name] <- raster::extract(layer, ZAF_bluecarbon_raw[,c("x", "y")])
}

ZAF_bluecarbon_final <- ZAF_bluecarbon_final %>% dplyr::select(-c("x", "y"))

In [ ]:
#Without Swartkops estuary
#ZAF_bluecarbon_final <- ZAF_bluecarbon_final %>% 
#                       dplyr::filter(estuary != "Swartkops" | Site != "Lower") 

In [ ]:
ref_numeric <- ZAF_bluecarbon_final %>% 
               dplyr::select_if(is.numeric) %>%
               dplyr::select(-c(mean_Corg, MgC_perHa)) %>%
               colnames()

Scaler <- caret::preProcess(ZAF_bluecarbon_final, method = list(center = ref_numeric, scale = ref_numeric))
ZAF_bluecarbon <- predict(Scaler, ZAF_bluecarbon_final)

In [ ]:
#https://stats.stackexchange.com/questions/59879/logistic-regression-anova-chi-square-test-vs-significance-of-coefficients-ano

In [ ]:
simple_lmer <- function(pred, response_var, random_fmla, data){
    
    fmla_p2 = paste(pred, random_fmla, sep = " + ")
    fmla = as.formula(paste(response_var, fmla_p2, sep = " ~ "))
    lm_fit <-  lmer(fmla, data = data, REML=FALSE)
    lmer_out <- broom::tidy(lm_fit)
    lmer_out$variable <- pred
     
    return(lmer_out)
}

In [ ]:
simple_lmer_anova <- function(pred, response_var, random_fmla, data){
    
    fmla_p2 = paste(pred, random_fmla, sep = " + ")
    fmla = as.formula(paste(response_var, fmla_p2, sep = " ~ "))
    lm_fit <-  lmer(fmla, data = data, REML=FALSE)
    lmer_anova <- car::Anova(lm_fit, test="Chisq") 
    return(lmer_anova)
}

In [ ]:
lmer_out <- lapply(ref_numeric, 
            simple_lmer, 
            response_var = "MgC_perHa",
            random_fmla = "(1|estuary:Site)",
            data = ZAF_bluecarbon_final)

In [ ]:
lmer_anova_out <-  lapply(ref_numeric, 
                   simple_lmer_anova, 
                   response_var = "MgC_perHa",
                   random_fmla = "(1|estuary:Site)",
                   data = ZAF_bluecarbon_final)

In [ ]:
(lmer_df <-  do.call(rbind, lmer_out) %>%
             data.frame() %>%
             dplyr::filter(effect == "fixed" & is.na(group) & term != "(Intercept)") %>%
             dplyr::select(variable, variable, p.value) %>% 
             mutate(signif = signif_annotate(p.value)) %>%
             arrange(variable) %>%
             rename(layer_code = variable))

In [ ]:
lmer_anova_df <-  do.call(rbind, lmer_anova_out) %>%
             data.frame() %>% tibble::rownames_to_column(var = "layer_code")
bioclim_results <- merge(lmer_anova_df, best_WC) %>% 
                   dplyr::select(description, Chisq, Df, Pr..Chisq.) %>% dplyr::rename(p_value = Pr..Chisq.)

In [ ]:
xtabs(~ Biogeographical_region + estuary, data = ZAF_bluecarbon_final)

In [ ]:
ZAF_bluecarbon_final$Biogeographical_region <- factor(ZAF_bluecarbon_final$Biogeographical_region, levels = c('Cool Temperate', 'Warm temperate','Subtropical'))

In [ ]:
ZAF_biogeograpy <- ZAF_bluecarbon_final %>%
                    group_by(Biogeographical_region) %>% 
                    rstatix::get_summary_stats(MgC_perHa) %>% 
                    dplyr::select(Biogeographical_region, n, mean, sd, se) %>%
                    dplyr::rename(n_Biogeography = n, mean_Biogeography = mean, sd_Biogeography = sd,  se_Biogeography = se)

In [ ]:
baseline_bio <- gls(MgC_perHa ~ 1, data = ZAF_bluecarbon_final, method="ML")
random_bio <- lme(MgC_perHa ~ 1, random = ~1|Site_ref, data = ZAF_bluecarbon_final, method="ML")
anova(baseline_bio, random_bio)

In [ ]:
fit_bio <- lmer(MgC_perHa ~  Biogeographical_region + (1|Site_ref),  data  = ZAF_bluecarbon_final)
(Bluecarbon_sites <- car::Anova(fit_bio, test="Chisq") %>%
          tibble::rownames_to_column(var = "description") %>%
          dplyr::rename(p_value = "Pr(>Chisq)"))

In [ ]:
(env_results <- rbind(Bluecarbon_sites, bioclim_results) %>% 
               dplyr::mutate(p_value = round(p_value, 4), Chisq = round(Chisq, 2)))
#print(xtable(env_results, digits=2), type = "html", file = "Table/env_stats.html", include.rownames=FALSE)
print(xtable(env_results, digits=7, type = "latex"), file = "Supplementary_material/env_stats.tex", include.rownames=FALSE)

In [ ]:
dim(ZAF_bluecarbon_final)

In [ ]:
glht_bio <- glht(fit_bio, linfct = mcp(Biogeographical_region = "Tukey"))
cld_bio <- multcomp::cld(glht_bio)
cld_bio <- cld_bio$mcletters$Letters %>% 
                 data.frame() %>%
                 setNames("cld_Biogeographical_region") %>%
                 tibble::rownames_to_column(var = "Biogeographical_region")

In [ ]:
(table_biogeography1 <- merge(ZAF_biogeograpy, cld_bio) %>%
                       dplyr::mutate(n_Biogeography = glue::glue("({n_Biogeography})")))

In [ ]:
anno_biogeography1 <-  ZAF_bluecarbon_final %>% 
                       dplyr::group_by(Biogeographical_region) %>%
                       dplyr::summarise(y1 = max(MgC_perHa) + 15,  y2 = y1 + 20) %>%
                       dplyr::ungroup() %>%
                       dplyr::mutate(x = row_number())

(biogeography_anno1 <-  merge(anno_biogeography1, table_biogeography1))

In [ ]:
p1 <- ggplot(data = ZAF_bluecarbon_final %>% 
             dplyr::rename(Estuary = estuary, `Sampling site` = Site)) +
      geom_boxplot(aes(y = MgC_perHa,
                      x = Biogeographical_region),
                      outlier.alpha = 0,
                      linewidth = 1) +
     geom_point(aes(y = MgC_perHa,
                x = Biogeographical_region,
                colour = Estuary,
                shape = `Sampling site`,
                fill = Estuary),
                size = 4,
                alpha = 0.5,
               position = "jitter") +
      labs(x  = "Biogeographical region",  y = expression(paste(C[org]~stocks~(Mg~C~ha^{-1})))) +
      scale_y_continuous(limits = c(0, 267), breaks = seq(0, 267, 25)) +
      theme(panel.grid.minor = element_blank(),
            panel.background = element_blank(),
            legend.box.background = element_rect(colour = "black"),
            legend.title = element_text(size=22, colour = "black"),
            legend.text = element_text(size=22, colour = "black"),
            axis.text.y = element_text(size=22, colour = "#434343"),
            axis.text.x = element_text(hjust = 1, size=22, colour = "#434343", angle = 45),
            axis.title = element_text(size=22, colour = "black"),
            panel.border = element_rect(colour = "black", fill=NA, linewidth=1)) +
      geom_text(data = biogeography_anno1,
               aes(label = n_Biogeography,
                   y = y1,
                   x = x),
                   size = 8,
                   alpha = 0.75,
                   colour = "#808080") +
      geom_text(data = biogeography_anno1,
               aes(label = cld_Biogeographical_region,
                   y = y2,
                   x = x),
                   size = 8,
                   colour = "#808080") +
      ggsci::scale_color_d3()

p1
ggsave("Figurex/biogeo.pdf", height = 10,  width = 7.5)

In [ ]:
######################################################################################################

                                  ####  REDUCED DATASET ####

######################################################################################################

In [ ]:
#Without Swartkops estuary
ZAF_bluecarbon_final2 <- ZAF_bluecarbon_final %>% 
                       dplyr::filter(estuary != "Swartkops" | Site != "Lower") 

In [ ]:
ZAF_biogeograpy2 <- ZAF_bluecarbon_final2 %>%
                    group_by(Biogeographical_region) %>% 
                    rstatix::get_summary_stats(MgC_perHa) %>% 
                    dplyr::select(Biogeographical_region, n, mean, sd, se) %>%
                    dplyr::rename(n_Biogeography = n, mean_Biogeography = mean, sd_Biogeography = sd,  se_Biogeography = se)

In [ ]:
ref_numeric <- ZAF_bluecarbon_final %>% 
               dplyr::select_if(is.numeric) %>%
               dplyr::select(-c(mean_Corg, MgC_perHa)) %>%
               colnames()

Scaler <- caret::preProcess(ZAF_bluecarbon_final2, method = list(center = ref_numeric, scale = ref_numeric))
ZAF_bluecarbon <- predict(Scaler, ZAF_bluecarbon_final2)

In [ ]:
#https://stats.stackexchange.com/questions/59879/logistic-regression-anova-chi-square-test-vs-significance-of-coefficients-ano

In [ ]:
simple_lmer <- function(pred, response_var, random_fmla, data){
    
    fmla_p2 = paste(pred, random_fmla, sep = " + ")
    fmla = as.formula(paste(response_var, fmla_p2, sep = " ~ "))
    lm_fit <-  lmer(fmla, data = data, REML=FALSE)
    lmer_out <- broom::tidy(lm_fit)
    lmer_out$variable <- pred
     
    return(lmer_out)
}

In [ ]:
simple_lmer_anova <- function(pred, response_var, random_fmla, data){
    
    fmla_p2 = paste(pred, random_fmla, sep = " + ")
    fmla = as.formula(paste(response_var, fmla_p2, sep = " ~ "))
    lm_fit <-  lmer(fmla, data = data, REML=FALSE)
    lmer_anova <- car::Anova(lm_fit, test="Chisq") 
    return(lmer_anova)
}

In [ ]:
lmer_out <- lapply(ref_numeric, 
            simple_lmer, 
            response_var = "MgC_perHa",
            random_fmla = "(1|estuary:Site)",
            data = ZAF_bluecarbon_final2)

In [ ]:
lmer_anova_out <-  lapply(ref_numeric, 
                   simple_lmer_anova, 
                   response_var = "MgC_perHa",
                   random_fmla = "(1|estuary:Site)",
                   data = ZAF_bluecarbon_final)

In [ ]:
(lmer_df <-  do.call(rbind, lmer_out) %>%
             data.frame() %>%
             dplyr::filter(effect == "fixed" & is.na(group) & term != "(Intercept)") %>%
             dplyr::select(variable, variable, p.value) %>% 
             mutate(signif = signif_annotate(p.value)) %>%
             arrange(variable) %>%
             rename(layer_code = variable))

In [ ]:
lmer_anova_df <-  do.call(rbind, lmer_anova_out) %>%
             data.frame() %>% tibble::rownames_to_column(var = "layer_code")
bioclim_results <- merge(lmer_anova_df, best_WC) %>% 
                   dplyr::select(description, Chisq, Df, Pr..Chisq.) %>% dplyr::rename(p_value = Pr..Chisq.)

In [ ]:
baseline_bio <- gls(MgC_perHa ~ 1, data = ZAF_bluecarbon_final, method="ML")
random_bio <- lme(MgC_perHa ~ 1, random = ~1|Site_ref, data = ZAF_bluecarbon_final, method="ML")
anova(baseline_bio, random_bio)

In [ ]:
fit_bio <- lmer(MgC_perHa ~  Biogeographical_region + (1|Site_ref),  data  = ZAF_bluecarbon_final2)
(Bluecarbon_sites <- car::Anova(fit_bio, test="Chisq") %>%
          tibble::rownames_to_column(var = "description") %>%
          dplyr::rename(p_value = "Pr(>Chisq)"))

In [ ]:
(env_results <- rbind(Bluecarbon_sites, bioclim_results) %>% 
               dplyr::mutate(p_value = round(p_value, 4), Chisq = round(Chisq, 2)))
#print(xtable(env_results, digits=2), type = "html", file = "Table/env_stats.html", include.rownames=FALSE)
print(xtable(env_results, digits=7, type = "latex"), file = "Supplementary_material/env_stats.tex", include.rownames=FALSE)

In [ ]:
glht_bio <- glht(fit_bio, linfct = mcp(Biogeographical_region = "Tukey"))
cld_bio <- multcomp::cld(glht_bio)
cld_bio <- cld_bio$mcletters$Letters %>% 
                 data.frame() %>%
                 setNames("cld_Biogeographical_region") %>%
                 tibble::rownames_to_column(var = "Biogeographical_region")

In [ ]:
table_biogeography2 <- merge(ZAF_biogeograpy2, cld_bio) %>%
                       dplyr::mutate(n_Biogeography = glue::glue("({n_Biogeography})"))

In [ ]:
anno_biogeography2 <-   ZAF_bluecarbon_final2 %>% 
                        dplyr::group_by(Biogeographical_region) %>%
                        dplyr::summarise(y1 = max(MgC_perHa) + 10,  y2 = y1 + 10) %>%
                        dplyr::ungroup() %>%
                        dplyr::mutate(x = row_number())

(biogeography_anno2 <-  merge(anno_biogeography2, table_biogeography2))

In [ ]:
biogeography_anno1

In [ ]:
p2 <- ggplot(data = ZAF_bluecarbon_final2 %>% dplyr::rename(Estuary = estuary)) +
      geom_boxplot(aes(y = MgC_perHa,
                      x = Biogeographical_region),
                      outlier.alpha = 0,
                      linewidth = 1) +
     geom_point(aes(y = MgC_perHa,
                x = Biogeographical_region,
                colour = Estuary,
                fill = Estuary),
                size = 4,
                alpha = 0.5,
               position = "jitter") +
      labs(x  = "Biogeographical region",  y = expression(paste(C[org]~stocks~(Mg~C~ha^{-1})))) +
      scale_y_continuous(limits = c(0, 150), breaks = seq(0, 125, 25)) +
      theme(panel.grid.minor = element_blank(),
            panel.background = element_blank(),
            legend.box.background = element_rect(colour = "black"),
            legend.title = element_text(size=22, colour = "black"),
            legend.text = element_text(size=22, colour = "black"),
            axis.text.y = element_text(size=22, colour = "#434343"),
            axis.text.x = element_text(hjust = 1, size=22, colour = "#434343", angle = 45),
            axis.title = element_text(size=22, colour = "black"),
            panel.border = element_rect(colour = "black", fill=NA, linewidth=1)) +
      geom_text(data = biogeography_anno2,
               aes(label = n_Biogeography,
                   y = y1,
                   x = x),
                   size = 8,
                   alpha = 0.75,
                   colour = "#808080") +
      geom_text(data = biogeography_anno2,
               aes(label = cld_Biogeographical_region,
                   y = y2,
                   x = x),
                   size = 8,
                   colour = "#808080") +
      ggsci::scale_color_d3()

In [ ]:
p2

In [ ]:
p1

In [ ]:
fit_bio2 <- lmer(MgC_perHa ~  Biogeographical_region + (1|Site_ref),  data  = ZAF_bluecarbon_final2)
summary(fit_bio2)

In [ ]:
(sample <- gmed_data[sample(10, 5), sample(54, 5)])